In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import NoSuchElementException

import undetected_chromedriver as uc
from webdriver_manager.chrome import ChromeDriverManager

# from bs4 import BeautifulSoup

import os
import time
import requests
import warnings
import unicodedata
import pandas as pd
from tqdm import tqdm
from random import randint

warnings.filterwarnings('ignore')

chrome_options = uc.ChromeOptions()

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-infobars');
chrome_options.add_argument("--safebrowsing-disable-download-protection")
chrome_options.add_argument("safebrowsing-disable-extension-blacklist")
chrome_options.add_argument("--disable-javascript")
chrome_options.add_argument("--start-maximized")
# Adding argument to disable the AutomationControlled flag 
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
 
# Exclude the collection of enable-automation switches 
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"]) 

# Turn-off userAutomationExtension 
chrome_options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

actions = ActionChains(driver)

In [2]:
def crawlPDF(company_number, cursor, number_of_companies):

    data = []

    driver.get(f"https://find-and-update.company-information.service.gov.uk/company/{company_number}/filing-history")

    time.sleep(3)

    driver.execute_script("window.scrollBy(0, 500);")  # Scrolls down 500px

    # Get company name and company number
    try:
        company_name = driver.find_element(By.CLASS_NAME, "heading-xlarge").text.strip().replace('\\', '').replace('/', '-')
        company_number = driver.find_element(By.ID, "company-number").text.strip()

        folder_path = f"RESULT/{company_number}_{company_name}"

        print(f"Processing company number: {cursor}/{number_of_companies} - {company_number} - {company_name}\n")

        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            tqdm.write(f"Folder created: {folder_path}")
        else:
            tqdm.write(f"Folder already exists: {folder_path}")

        # Filter for accounts
        checkbox = driver.find_element(By.ID, "filter-category-accounts")
        if not checkbox.is_selected():
            # Click the checkbox to select it
            checkbox.click()
    except NoSuchElementException:
        tqdm.write(f"Company {company_number} not found or does not have a valid filing history page.")
        return []

    # Loop through pages until there are no more pages to scrape
    page_number = 1 
    while True:
        try:

            time.sleep(2)

            table = driver.find_element(By.ID, "fhTable")

            rows = table.find_element(By.TAG_NAME, "tbody").find_elements(By.TAG_NAME, "tr")
            for row in tqdm(rows[1:], desc=f"Scraping page {page_number}"):  # Skip the header row
                columns = row.find_elements(By.TAG_NAME, "td")
                if len(columns) > 0:
                    date = columns[0].text.strip()
                    description = columns[2].text.strip()
                    if ("Full accounts".lower() in description.lower()) and "Request Document" not in columns[3].find_element(By.TAG_NAME, "a").text.strip() :
                        link = columns[3].find_element(By.TAG_NAME, "a").get_attribute("href")
                    else:
                        link = 'None'

                    if link != 'None':
                        while True:
                            response = requests.get(link)
                            if response.status_code == 200:
                                with open(f"{folder_path}/{date}_{description}.pdf", "wb") as f:
                                    f.write(response.content)
                                break
                            else:
                                tqdm.write(f"Failed to download. Status code: {response.status_code}")
                                tqdm.write("Retrying in 5 seconds...")
                                time.sleep(5)
                    
                    data.append({
                        'Company Name': company_name,
                        'Company Number': company_number,
                        'Date': date,
                        'Description': description,
                        'Link': link
                    })

            
            time.sleep(2)
            next_page = driver.find_element(By.CLASS_NAME, "govuk-pagination__next")
            next_page.click()
            tqdm.write("Next page clicked.")
            page_number += 1

        except:
            tqdm.write("No more pages to scrape.\n\n")
            break

    # Save the data to a CSV file
    pd.DataFrame(data).to_csv(f"{folder_path}/Filling_History_{company_number}_{company_name}.csv", index=False)

    return data


FULL_DATA = []
cursor = 1
company_list = pd.read_excel('JPG-to-excel_vpM.xlsx')
company_number_list = company_list['company_number']
for company in company_number_list:
    try:
        FULL_DATA.extend(crawlPDF(company, cursor, len(company_number_list)))
        cursor += 1
    except Exception as e:
        tqdm.write(f"Error processing company {company}: {e}")
        continue

pd.DataFrame(FULL_DATA).to_csv('RESULT/Filling_History.csv', index=False)

Processing company number: 1/25 - Company number 02719242 - SONY PICTURES TELEVISION UK RIGHTS LIMITED

Folder created: RESULT/Company number 02719242_SONY PICTURES TELEVISION UK RIGHTS LIMITED


Scraping page 1: 100%|██████████| 25/25 [01:04<00:00,  2.58s/it]


Next page clicked.


Scraping page 2: 100%|██████████| 12/12 [00:22<00:00,  1.91s/it]


No more pages to scrape.


Processing company number: 2/25 - Company number 02829355 - SONY INTERACTIVE ENTERTAINMENT UK LIMITED

Folder created: RESULT/Company number 02829355_SONY INTERACTIVE ENTERTAINMENT UK LIMITED


...

No more pages to scrape.


Processing company number: 24/25 - Company number 01471066 - SONY MUSIC ENTERTAINMENT UK LIMITED

Folder created: RESULT/Company number 01471066_SONY MUSIC ENTERTAINMENT UK LIMITED


Scraping page 1: 100%|██████████| 25/25 [01:20<00:00,  3.24s/it]


Next page clicked.


Scraping page 2: 100%|██████████| 25/25 [00:39<00:00,  1.57s/it]


No more pages to scrape.


Processing company number: 25/25 - Company number OE029601 - SONY EUROPE B.V.

Folder created: RESULT/Company number OE029601_SONY EUROPE B.V.
No more pages to scrape.


